In [1]:
!git clone https://github.com/openai/CLIP.git
# CLIP: Contrastive Language-Image Pre-Training
# Learning Transferable Visual Models From Natural Language Supervision (paper)

!git clone https://github.com/CompVis/taming-transformers.git
# Taming Transformers for High-Resolutions Image Synthesis (paper)

Cloning into 'CLIP'...
remote: Enumerating objects: 256, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 256 (delta 126), reused 110 (delta 110), pack-reused 102 (from 1)
Receiving objects: 100% (256/256), 8.86 MiB | 14.14 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Cloning into 'taming-transformers'...
remote: Enumerating objects: 1342, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1342 (delta 0), reused 0 (delta 0), pack-reused 1341 (from 2)
Receiving objects: 100% (1342/1342), 409.77 MiB | 23.27 MiB/s, done.
Resolving deltas: 100% (282/282), done.


In [2]:
# install some extra libraries

!pip install --no-deps ftfy regex tqdm  # install text fixing, regex, and progress bar libraries without dependencies
!pip install omegaconf==2.0.0 pytorch-lightning==1.0.8  # install specific versions for configuration and training framework
!pip uninstall torchtext --yes  # remove torchtext as it may conflict or is not needed
!pip install einop  # install einop, a lightweight wrapper for Einstein summation operations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [4]:
# import libraries

import numpy as np  # numerical operations and arrays
import torch, os, imageio, pdb, math  # torch for deep learning, os for file paths, imageio for reading/writing images, pdb for debugging, math for mathematical functions
import torchvision  # PyTorch vision utilities
import torchvision.transforms as T  # image transformations
import torchvision.transforms.functional as TF  # functional image transformations

import PIL  # Python Imaging Library for handling images
import matplotlib.pyplot as plt  # plotting and visualisation

import yaml  # for reading YAML configuration files
from omegaconf import OmegaConf  # structured configuration management

from CLIP import clip  # import CLIP model (Contrastive Language–Image Pretraining)

# import warnings
# warnings.filterwarnings('ignore')  # (optional) suppress warnings


In [5]:
# helper functions

def show_from_tensor(tensor):
  img = tensor.clone()  # create a copy of the tensor
  img = img.mul(255).byte()  # scale pixel values from [0,1] to [0,255] and convert to byte type
  img = img.cpu().numpy().transpose((1, 2, 0))  # move to CPU and change to H x W x C format for plotting

  plt.figure(figsize=(10, 7))  # set figure size
  plt.axis('off')  # remove axes for clean display
  plt.imshow(img)  # show image
  plt.show()

def norm_data(data):
  return (data.clip(-1, 1) + 1) / 2  # clip values to [-1, 1] and normalise to [0, 1] range

# parameters

learning_rate = 0.5  # learning rate for the optimiser
batch_size = 1  # number of samples per training iteration
wd = 0.1  # weight decay (regularisation parameter)
noise_factor = 0.1  # scale of noise added to the image

total_iter = 100  # total number of optimisation steps
im_shape = [225, 400, 3]  # image dimensions: height, width, channels
size1, size2, channels = im_shape  # unpack image dimensions


In [6]:
### clip model ###

clipmodel, _ = clip.load("ViT-B/32", jit = False)  # load the CLIP model with Vision Transformer (ViT-B/32) architecture
clipmodel.eval()  # set the model to evaluation mode
print(clip.available_models())  # print all available CLIP model variants

print("CLIP model visual input resolution: ", clipmodel.visual.input_resolution)  # display expected image resolution

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # choose GPU if available, else CPU
torch.cuda.empty_cache()  # clear unused GPU memory

100%|███████████████████████████████████████| 338M/338M [00:22<00:00, 15.9MiB/s]


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']
Clip model visual input resolution:  224
